In [ ]:
!pip -q install langchain transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.7/823.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## StarChatAlpha




In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Load the model


In [ ]:
import torch
import transformers
from transformers import GenerationConfig, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/starchat-alpha")

model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/starchat-alpha",
                                              load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                             use_auth_token= True
                                              )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tokenizer.eos_token, tokenizer.pad_token

('<|endoftext|>', '<|endoftext|>')

In [ ]:
tokenizer.eos_token_id, tokenizer.pad_token_id

(0, None)

In [ ]:
tokenizer.pad_token_id = 0

### Basic Text generation

In [ ]:

full_prompt

'<|system|>\nBelow is a conversation between a human user and a helpful AI coding assistant.<|end|>\n<|user|>\nWrite a python function that reverses an input string<|end|>\n<|assistant|>'

In [ ]:
import textwrap

def generate_response(input_prompt):
    system_prompt = "<|system|>\nBelow is a conversation between a human user and a helpful AI coding assistant.<|end|>\n"

    user_prompt = f"<|user|>\n{input_prompt}<|end|>\n"

    assistant_prompt = "<|assistant|>"

    full_prompt = system_prompt + user_prompt + assistant_prompt

    inputs = tokenizer.encode(full_prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(inputs,
                            eos_token_id = 0,
                            pad_token_id = 0,
                            max_length=256,
                            early_stopping=True)
    output =  tokenizer.decode(outputs[0])
    output = output[len(full_prompt):]
    if "<|end|>" in output:
        cutoff = output.find("<|end|>")
        output = output[:cutoff]
    print(input_prompt+'\n')
    # wrapped_text = textwrap.fill(output, width=100)
    # print(wrapped_text +'\n\n')
    print(output)


In [ ]:
generate_response("Write a python function that reverses each word in an input string")

Write a python function that reverses each word in an input string


Here's a Python function that reverses each word in an input string:

def reverse_words(input_str):
    words = input_str.split()
    reversed_words = [word[::-1] for word in words]
    return " ".join(reversed_words)

This function takes an input string and splits it into a list of words using the split() method. It then uses a list comprehension to reverse each word in the list using the [::-1] slicing notation. Finally, the reversed words are joined together using the join() method and the result is returned.


In [ ]:
generate_response("What is Flask?")

What is Flask?


Flask is a micro web framework written in Python. It is classified as a microframework because it does not require particular tools or libraries. It has no database abstraction layer, form validation, or any other components where pre-existing third-party libraries provide common functions. However, Flask supports extensions that can add application features as if they were implemented in Flask itself.


In [ ]:
generate_response("def scrape_url(url):")

def scrape_url(url):


def scrape_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # extract data from soup and return it


In [ ]:
generate_response("write a JS function that sorts a list alphabetically")

write a JS function that sorts a list alphabetically


function sortListAlphabetically(list) {
  list.sort();
}
